In [1]:
!pip install transformers
!pip install datasets
!pip install librosa

   ---------------------------------------- 0.0/260.1 kB ? eta -:--:--
   ----------------------- ---------------- 153.6/260.1 kB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 260.1/260.1 kB 4.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/64.6 kB ? eta -:--:--
   ---------------------------------------- 64.6/64.6 kB ? eta 0:00:00
   ---------------------------------------- 0.0/184.7 kB ? eta -:--:--
   --------------------------------------- 184.7/184.7 kB 10.9 MB/s eta 0:00:00


In [1]:
model_checkpoint = "facebook/wav2vec2-conformer-rel-pos-large"
batch_size = 64

In [2]:
ls

 Volume in drive D is New Volume
 Volume Serial Number is 6220-5780

 Directory of D:\Freelancing\AI Feature Scoring\training data

29-06-2024  16:28    <DIR>          .
29-06-2024  16:28    <DIR>          ..
29-06-2024  15:51    <DIR>          .ipynb_checkpoints
25-06-2024  16:35            25,438 ai_feature_scoring_speech_dataset.xlsx
13-05-2024  16:22           147,804 audio_0a2fb866-25a3-42e0-b672-a02f6b8ddd1a.m4a
29-06-2024  16:07    <DIR>          audio_files
29-06-2024  16:11    <DIR>          audio_files_in_mp3
29-06-2024  16:28            39,956 Generating Feature Vector for Speeches.ipynb
28-06-2024  15:51             1,269 Preprocess Audio.ipynb
26-06-2024  17:45             1,312 preprocessing.py
26-06-2024  18:19            27,334 processed_audio_sample_scoring.xlsx
               6 File(s)        243,113 bytes
               5 Dir(s)  922,854,739,968 bytes free


In [3]:
import numpy as np 
import pandas as pd

df = pd.read_excel("processed_audio_sample_scoring.xlsx")

In [4]:
df.head()

,Unnamed: 0,Record Audio Name,Content,Pronun,Fluency,feature_vector
0,0,audio_beb45dff-f264-43c5-b051-61dab6a3b2a1,18,11,25,NaN
1,1,audio_f86da9b5-9ed2-4153-9508-e59448e7d8bc,70,53,64,NaN
2,2,audio_0e7cded9-1569-4202-98f8-59f88384437b,54,40,51,NaN
3,3,audio_08bf8a53-1e31-4d82-8979-b53a14baa196,28,14,36,NaN
4,4,audio_a3b474a9-3b8e-4533-aa95-0cda51f1395d,16,11,25,NaN


In [5]:
'''
Transformer's FeatureExtractor will normalize the inputs and put 
them in a format the model expects, as well as generate the other inputs that 
the model requires
'''

"\nTransformer's FeatureExtractor will normalize the inputs and put \nthem in a format the model expects, as well as generate the other inputs that \nthe model requires\n"

In [6]:
# from transformers import AutoFeatureExtractor

# feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)

In [9]:
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "Wav2Vec2Processor",
  "return_attention_mask": true,
  "sampling_rate": 16000
}

In [7]:
max_duration = 40

In [11]:
def preprocess_function(examples):
    audio_arrays = [x['array'] for x in examples['audio']]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate = feature_extractor.sampling_rate,
        max_length = int(feature_extractor.sampling_rate * max_duration),
        truncation = True
    )
    
    return inputs

In [12]:
for index, row in df.iterrows():
    print(index)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
import librosa
import torch
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model

In [18]:
ls

 Volume in drive D is New Volume
 Volume Serial Number is 6220-5780

 Directory of D:\Freelancing\AI Feature Scoring\training data

29-06-2024  16:25    <DIR>          .
29-06-2024  16:25    <DIR>          ..
29-06-2024  15:51    <DIR>          .ipynb_checkpoints
25-06-2024  16:35            25,438 ai_feature_scoring_speech_dataset.xlsx
13-05-2024  16:22           147,804 audio_0a2fb866-25a3-42e0-b672-a02f6b8ddd1a.m4a
29-06-2024  16:07    <DIR>          audio_files
29-06-2024  16:11    <DIR>          audio_files_in_mp3
29-06-2024  16:25            39,076 Generating Feature Vector for Speeches.ipynb
28-06-2024  15:51             1,269 Preprocess Audio.ipynb
26-06-2024  17:45             1,312 preprocessing.py
26-06-2024  18:19            27,334 processed_audio_sample_scoring.xlsx
               6 File(s)        242,233 bytes
               5 Dir(s)  922,854,780,928 bytes free


In [32]:
os.path.isfile("audio_files//audio_beb45dff-f264-43c5-b051-61dab6a3b2a1.m4a")

True

In [9]:
path = "audio_files//audio_fc49c5b8-0d30-4f5a-9cae-4ed06e68149d.m4a"
input_audio, sample_rate = librosa.load(path,  sr=16000)

C:\Users\maste\AppData\Local\Temp\ipykernel_1744\957617493.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  input_audio, sample_rate = librosa.load(path,  sr=16000)
C:\Users\maste\anaconda3\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [10]:
model_name = "facebook/wav2vec2-large-xlsr-53"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
model = Wav2Vec2Model.from_pretrained(model_name)

i= feature_extractor(input_audio, return_tensors="pt", sampling_rate=sample_rate)
# with torch.no_grad():
#   o= model(i.input_values)
# print(o.keys())
# print(o.last_hidden_state.shape)
# print(o.extract_features.shape)

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

C:\Users\maste\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\maste\.cache\huggingface\hub\models--facebook--wav2vec2-large-xlsr-53. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


odict_keys(['last_hidden_state', 'extract_features'])
torch.Size([1, 252, 1024])
torch.Size([1, 252, 512])


In [19]:
o.extract_features

tensor([[[ 0.6918,  0.3132, -1.4722,  ..., -0.4782, -1.0847,  1.3629],
         [ 0.6918,  0.3132, -1.4722,  ..., -0.4782, -1.0847,  1.3629],
         [ 0.0926, -0.5830, -1.5916,  ..., -0.1743, -0.6749,  0.7679],
         ...,
         [ 0.5284, -0.9827,  1.2669,  ...,  0.9075, -1.1282, -0.3290],
         [ 0.7884, -1.0612,  1.3367,  ..., -0.1114, -1.2140, -0.3555],
         [-0.2587, -1.0298,  1.5370,  ...,  0.0839, -1.2938, -0.8687]]])

In [14]:
type(i)

transformers.feature_extraction_utils.BatchFeature

In [15]:
i

{'input_values': tensor([[0.0003, 0.0003, 0.0003,  ..., 0.2625, 0.2615, 0.0003]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32)}

In [36]:
model_name = "facebook/wav2vec2-large-xlsr-53"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
model = Wav2Vec2Model.from_pretrained(model_name)

def get_feature_vector(path):
    print(f"Loading audio from path: {path}")
    
    input_audio, sample_rate = librosa.load(path, sr=16000)
    print("Audio loaded successfully.")
    print(f"Sample rate: {sample_rate}")
    print(f"Input audio shape: {input_audio.shape}")
    
    print("Extracting features from the audio.")
    features = feature_extractor(input_audio, return_tensors="pt", sampling_rate=sample_rate)
    print("Features extracted successfully.")
    print(f"Feature shape: {features.input_values.shape}")
    
    print("Passing features through the model.")
    with torch.no_grad():
        output = model(features.input_values)
    
    print("Model inference completed.")
    print(f"Output keys: {output.keys()}")
    print(f"Last hidden state shape: {output.last_hidden_state.shape}")
    print(f"Extracted features shape: {output.extract_features.shape}")

    print("************" * 50)
    print("************" * 50)
    print("************" * 50)

    return output.extract_features

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
df.columns

Index(['Unnamed: 0', 'Record Audio Name', 'Content', 'Pronun', 'Fluency',
       'feature_vector'],
      dtype='object')

In [39]:
import os

for index, row in df.iterrows():
    audio_mp4a = "audio_files\\" + row["Record Audio Name"] + ".m4a"
    audio_mp3 = "audio_files\\" + row["Record Audio Name"] + ".mp3"
    audio_wav = "audio_files\\" + row["Record Audio Name"] + ".wav"

    print(audio_mp4a)
    if(os.path.isfile(audio_mp4a)):
        feature_vector = get_feature_vector(audio_mp4a)
        df.at[index, "feature_vector"] = feature_vector

    elif(os.path.isfile(audio_mp3)):
        feature_vector = get_feature_vector(audio_mp3)
        df.at[index, "feature_vector"] = feature_vector

    elif(os.path.isfile(audio_wav)):
        feature_vector = get_feature_vector(audio_wav)
        df.at[index, "feature_vector"] = feature_vector
    else:
        print("bruh")

audio_files\audio_beb45dff-f264-43c5-b051-61dab6a3b2a1.m4a
Loading audio from path: audio_files\audio_beb45dff-f264-43c5-b051-61dab6a3b2a1.m4a
Audio loaded successfully.
Sample rate: 16000
Input audio shape: (115543,)
Extracting features from the audio.
Features extracted successfully.
Feature shape: torch.Size([1, 115543])
Passing features through the model.


C:\Users\maste\AppData\Local\Temp\ipykernel_1744\3100802130.py:8: UserWarning: PySoundFile failed. Trying audioread instead.
  input_audio, sample_rate = librosa.load(path, sr=16000)


Model inference completed.
Output keys: odict_keys(['last_hidden_state', 'extract_features'])
Last hidden state shape: torch.Size([1, 360, 1024])
Extracted features shape: torch.Size([1, 360, 512])
************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************
*********************************************************************************************************************************************************************************************************

NameError: name 'value' is not defined